## Chains

The chains are responsible for creating an end-to-end pipeline for using the language models. They will join the model, prompt, memory, parsing output, and debugging capability and provide an easy-to-use interface. 

A chain will 
+ receive the user’s query as an input, 
+ process the LLM’s response, and lastly,
+ return the output to the user.

It is possible to design a custom pipeline by inheriting the Chain class. For example, the LLMChain is the simplest form of chain in LangChain, inheriting from the Chain parent class. We will start by going through ways to invoke this class and follow it by looking at adding different functionalities.

In [1]:
# !pip install langchain==0.0.208 deeplake openai==0.27.8 tiktoken scikit-learn deeplake

In [2]:
# Load environment variables
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

True

### LLMChain

In [3]:
from langchain import PromptTemplate, OpenAI, LLMChain

prompt_template = "What is a word to replace the following: {word}?"

# Set the "OPENAI_API_KEY" environment variable before running following line.
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0)

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)

In [4]:
llm_chain("artificial")

{'word': 'artificial', 'text': '\n\nSynthetic'}

In [5]:
#It is also possible to use the .apply() method to pass multiple inputs at once and receive a list for each input. 
input_list = [
    {"word": "artificial"},
    {"word": "intelligence"},
    {"word": "robot"}
]

llm_chain.apply(input_list)

[{'text': '\n\nSynthetic'},
 {'text': '\n\nCleverness'},
 {'text': '\n\nandroid'}]

In [6]:
#The .generate() method will return an instance of LLMResult, which provides more information. For example, the finish_reason key indicates the reason behind the stop of the generation process. It could be stopped, meaning the model decided to finish or reach the length limit. There is other self-explanatory information like the number of total used tokens or the used model.
llm_chain.generate(input_list)

LLMResult(generations=[[Generation(text='\n\nSynthetic', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nCleverness', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nandroid', generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {'total_tokens': 42, 'prompt_tokens': 33, 'completion_tokens': 9}, 'model_name': 'gpt-3.5-turbo-instruct'}, run=[RunInfo(run_id=UUID('08cddb52-73e8-4410-864e-3b03e915c01c')), RunInfo(run_id=UUID('39a12341-8a70-498b-9589-71d02cc8b92d')), RunInfo(run_id=UUID('5448796e-b783-4d12-bb3a-ead4c9d48344'))])

In [7]:
#The next method we will discuss is .predict() (which could be used interchangeably with .run()).
#Its best use case is to pass multiple inputs for a single prompt. However, it is possible to use it with one input variable as well. The following prompt will pass both the word we want a substitute for and the context the model must consider.
prompt_template = "Looking at the context of '{context}'. What is an appropriate word to replace the following: {word}?"

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(template=prompt_template, input_variables=["word", "context"]))

llm_chain.predict(word="fan", context="object")
# or llm_chain.run(word="fan", context="object")

'\n\nSupporter'

In [8]:
llm_chain.predict(word="fan", context="humans")
# or llm_chain.run(word="fan", context="humans")

'\n\nSupporter'

### Parsers

In [9]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
template = """List all possible words as substitute for 'artificial' as comma separated."""

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(template=template, output_parser=output_parser, input_variables=[]),
    output_parser=output_parser)

llm_chain.predict()

['synthetic',
 'man-made',
 'fake',
 'imitation',
 'simulated',
 'faux',
 'fabricated',
 'counterfeit',
 'ersatz',
 'false',
 'pretend',
 'mock',
 'spurious',
 'bogus',
 'phony',
 'contrived',
 'manufactured',
 'unnatural',
 'inauthentic']

## Conversational Chain (Memory)

Depending on the application, memory is the next component that will complete a chain. LangChain provides a ConversationalChain to track previous prompts and responses using the ConversationalBufferMemory class.

In [10]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

output_parser = CommaSeparatedListOutputParser()
conversation = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

conversation.predict(input="List all possible words as substitute for 'artificial' as comma separated.")

' Synthetic, simulated, man-made, fake, imitation, fabricated, manufactured, engineered, virtual, computer-generated, digital, non-natural, inauthentic, contrived, ersatz, pseudo, phony, false, unreal, pretend, counterfeit, spurious, bogus, sham, feigned, deceptive, fraudulent, specious, fallacious, misleading, delusive, illusory, fictitious, make-believe, imaginary, unreal, fictive, fanciful, fantastic, mythical, legendary, mythological, fabricated, created, contrived, invented, made-up, unreal, imaginary, fictional, mythical, legendary, mythological, fanciful, fantastic, make-believe, pretend, simulated, virtual, computer-generated, digital, non-natural, inauthentic, ersatz, pseudo, phony, false, counterfeit, spurious, bogus, sham, feigned, deceptive, fraudulent, specious, fallacious, misleading, delusive, illusory, fictitious, make-believe, imaginary, unreal, fictive, fanciful, fantastic, mythical, legendary, mythological, fabricated, created, contrived, invented, made-up, unreal, i

In [11]:
#Now, we can ask it to return the following four replacement words. It uses the memory to find the next options.
conversation.predict(input="And the next 4?")

' Imagined, conceptual, conceptualized, theoretical.'

## Sequential Chain

Another helpful feature is using a sequential chain that concatenates multiple chains into one. The following code shows a sample usage.

In [12]:
# poet
poet_template: str = """You are an American poet, your job is to come up with\
poems based on a given theme.

Here is the theme you have been asked to generate a poem on:
{input}\
"""

poet_prompt_template: PromptTemplate = PromptTemplate(
    input_variables=["input"], template=poet_template)

# creating the poet chain
poet_chain: LLMChain = LLMChain(
    llm=llm, output_key="poem", prompt=poet_prompt_template)

# critic
critic_template: str = """You are a critic of poems, you are tasked\
to inspect the themes of poems. Identify whether the poem includes romantic expressions or descriptions of nature.

Your response should be in the following format, as a Python Dictionary.
poem: this should be the poem you received 
Romantic_expressions: True or False
Nature_descriptions: True or False

Here is the poem submitted to you:
{poem}\
"""

critic_prompt_template: PromptTemplate = PromptTemplate(
    input_variables=["poem"], template=critic_template)

# creating the critic chain
critic_chain: LLMChain = LLMChain(
    llm=llm, output_key="critic_verified", prompt=critic_prompt_template)

In [13]:
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[poet_chain, critic_chain])

In [14]:
# Run the poet and critic chain with a specific theme
theme: str = "the beauty of nature"
review = overall_chain.run(theme)

# Print the review to see the critic's evaluation
print(review)



poem: Nature's Beauty
Romantic_expressions: False
Nature_descriptions: True


### Debug

It is possible to trace the inner workings of any chain by setting the verbose argument to True. As you can see in the following code, the chain will return the initial prompt and the output. The output depends on the application. It may contain more information if there are more steps.

In [15]:
template = """List all possible words as substitute for 'artificial' as comma separated.

Current conversation:
{history}

{input}"""

conversation = ConversationChain(
    llm=llm,
    prompt=PromptTemplate(template=template, input_variables=["history", "input"], output_parser=output_parser),
    memory=ConversationBufferMemory(),
    verbose=True)

conversation.predict(input="")



> Entering new ConversationChain chain...
Prompt after formatting:
List all possible words as substitute for 'artificial' as comma separated.

Current conversation:




> Finished chain.


'\n1. Synthetic\n2. Man-made\n3. Faux\n4. Imitation\n5. Simulated\n6. Fake\n7. Replicated\n8. Manufactured\n9. Counterfeit\n10. Mock\n11. Pretend\n12. False\n13. Inauthentic\n14. Unnatural\n15. Fabricated'

## Custom Chain

The LangChain library has several predefined chains for different applications like Transformation Chain, LLMCheckerChain, LLMSummarizationCheckerChain, and OpenAPI Chain, which all share the same characteristics mentioned in previous sections. It is also possible to define your chain for any custom task. In this section, we will create a chain that returns a word's meaning and then suggests a replacement.

It starts by defining a class that inherits most of its functionalities from the Chain class. Then, the following three methods must be declared depending on the use case. The `input_keys` and `output_keys` methods let the model know what it should expect, and a `_call` method runs each chain and merges their outputs.

In [16]:
from langchain.chains import LLMChain
from langchain.chains.base import Chain

from typing import Dict, List

class ConcatenateChain(Chain):
    chain_1: LLMChain
    chain_2: LLMChain

    @property
    def input_keys(self) -> List[str]:
        # Union of the input keys of the two chains.
        all_input_vars = set(self.chain_1.input_keys).union(set(self.chain_2.input_keys))
        return list(all_input_vars)

    @property
    def output_keys(self) -> List[str]:
        return ['concat_output']

    def _call(self, inputs: Dict[str, str]) -> Dict[str, str]:
        output_1 = self.chain_1.run(inputs)
        output_2 = self.chain_2.run(inputs)
        return {'concat_output': output_1 + output_2}

In [17]:
#Then, we will declare each chain individually using the LLMChain class.
prompt_1 = PromptTemplate(
    input_variables=["word"],
    template="What is the meaning of the following word '{word}'?",
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

prompt_2 = PromptTemplate(
    input_variables=["word"],
    template="What is a word to replace the following: {word}?",
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

#Lastly, we call our custom chain ConcatenateChain to merge the results of the chain_1 and chain_2
concat_chain = ConcatenateChain(chain_1=chain_1, chain_2=chain_2)
concat_output = concat_chain.run("artificial")
print(f"Concatenated output:\n{concat_output}")

Concatenated output:


Artificial means something that is made or produced by humans, rather than occurring naturally. It can also refer to something that is not genuine or authentic, but rather created or imitated to appear real.

Synthetic
